**Inputs**
* NEX-GDDP data
* Hazard definition(s)

**Outputs**
* ImageCollection(s) containing hazard magnitudes for each year-model pair
* Stored in users/tedwongwri/dataportal/magnitudes/HAZARDNAME
* **You must manually create the empty ImageCollection** in the dataportal/magnitudes directory before trying to export results

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWhhC_bUbdBIt63tOmbrtosCa4v8s_ShtslSM1OVyQ0jr1p4btvgDlQ

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [58]:
from datetime import date, timedelta
from calendar import isleap

In [59]:
# Define hazards here

def dryspells(yeardata):
    dry_days = yeardata.map(lambda d: d.eq(0).multiply(1))
    return process_runs(dry_days, 5, 'count')

def ece(yeardata):
    return

def ehe(yeardata):
    tasmax_p99_model = TASMAX_99.filterMetadata('model', 'equals', model).first()
    over_year = yeardata.map(lambda x: x.gt(tasmax_p99_model).multiply(1))
    return over_year.sum()

def mtt35(yeardata):
    over_year = yeardata.map(lambda x: x.gt(35 + 273.15).multiply(1))
    return over_year.sum()

def maxdryspell(yeardata):
    # Climdex CDD: Let RR_ij be the daily precipitation amount on day i in period j. Count the largest number of consecutive days where RR_ij < 1mm
    dry_days = yeardata.map(lambda d: d.multiply(86400).eq(0))
    return process_runs(dry_days, 1, 'max')

susc = ee.Image("users/tedwongwri/dataportal/landslide/susc")
ari95 = ee.Image("users/tedwongwri/dataportal/landslide/ARI95")
def modlandslide(yeardata):
# Returns number of days in year with moderate or high landslide risk
# antecedent rainfall index from https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017EF000715
    start_date, end_date = date(year, 1, 1), date(year, 12, 31)
    modeldata = NEX.filterDate((start_date - timedelta(days=7)).isoformat(), end_date.isoformat()).filterMetadata('model', 'equals', model).select('pr')
    modrisk_list = ee.List([]) # holds results
    highrisk_list = ee.List([]) # holds results
    precip_list = ee.List([ee.Image(0)]) # holds input precip for current date and six previous; zero image is dummy, will be removed immediately
    for t in range(6):  # Create list with seven days of data, with most recent last
        tt = 6 - t
        tdate = start_date - timedelta(days=tt)
        modelscenario_data = modeldata.filterMetadata('scenario', 'equals', ['historical', 'rcp85'][(tdate.year > 2005) * 1])
        precip_list = precip_list.add(modelscenario_data.filterDate(tdate.isoformat(), (tdate + timedelta(days=1)).isoformat()).first().multiply(86400))
    for datediff in range((end_date - start_date).days + ([0, -1][(isleap(year) and model in NOLEAP_MODELS) * 1])):
        focal_date = skipleap(start_date + timedelta(days = datediff), model)
        modelscenario_data = modeldata.filterMetadata('scenario', 'equals', ['historical', 'rcp85'][(focal_date.year > 2005) * 1])
        precip_list = precip_list.slice(1, 7)  #Every day remove one, add one
        precip_list = precip_list.add(modelscenario_data.filterDate(focal_date.isoformat(), (skipleap(focal_date + timedelta(days=1), model)).isoformat()).first().multiply(86400))
        numerator = ee.Image.constant(0)
        denominator = ee.Image.constant(0)
        for t in range(7):
            tdate = skipleap(focal_date - timedelta(days=t), model)
            precip = ee.Image(precip_list.get(-(t + 1)))
            w = 1.0 / ((t + 1)**2)
            numerator = numerator.add(precip.multiply(w))
            denominator = denominator.add(w)
        modrisk_list = modrisk_list.add((numerator.divide(denominator).gt(ari95).multiply(1)).multiply(susc.gt(2).multiply(1)))
        highrisk_list = highrisk_list.add((numerator.divide(denominator).gt(ari95).multiply(1)).multiply(susc.gt(4).multiply(1)))
    return ee.ImageCollection(modrisk_list).sum()
    
    

hazards = [
            {'definition': ehe,
             'variable': 'tasmax'
            },
            {'definition': mtt35,
             'variable': 'tasmax'
            },
            {'definition': modlandslide,
             'variable': 'pr'
            },
]

In [3]:
def process_runs(imgs, runLength, resultType): 
# from Logan Byers
    def doOne(img, data):
   #data = ee.Image(data);

        dataDict = ee.Dictionary(data)

        previousThresholdImage = ee.Image(dataDict.get('previousThresholdImage'))
        currentStreakImage = ee.Image(dataDict.get('currentStreakImage')).uint16()
        streakCountImage = ee.Image(dataDict.get('streakCountImage')).uint16()
        longestStreakImage = ee.Image(dataDict.get('longestStreakImage')).uint16()
        streakAccumulation = ee.Image(dataDict.get('streakAccumulation')).uint16()
        numRemaining = ee.Number(dataDict.get('numRemaining'))

        # WHERE yesterday AND today : 1, else 0
        #continueStreakImage = previousThresholdImage.and(ee.Image(img))
        continueStreakImage = previousThresholdImage.multiply(ee.Image(img)).multiply(ee.Image(numRemaining).gt(1).multiply(1))

        # WHERE NOT on streak :  yesterday streak length, else 0
        #streakEndedImage = currentStreakImage.multiply(currentStreakImage.and(continueStreakImage.not()))
        streakEndedImage = currentStreakImage.multiply(currentStreakImage.multiply(continueStreakImage.multiply(-1).add(1)))

        # WHERE NOT on streak AND yesterday streak length > length threshold : 1, else 0
        endedStreakExceedsLengthImage = currentStreakImage.multiply(streakEndedImage).gte(runLength)

        # update the state
        accumulator = ee.Dictionary.fromLists([
            'previousThresholdImage',
            'currentStreakImage',
            'streakCountImage',
            'longestStreakImage',
            'streakAccumulation',
            'numRemaining'
          ], [
            # previousThresholdImage --> today's image
            ee.Image(img),
            # currentStreakImage --> today's image PLUS yesterday's streak (where continuing)
            currentStreakImage
              .multiply(continueStreakImage).add(ee.Image(img)),
            # streakCountImage --> PLUS 1 where long streak ended today
            streakCountImage.add(endedStreakExceedsLengthImage),
            # longestStreakImage --> larger of prev and current value
            longestStreakImage.max(currentStreakImage.multiply(continueStreakImage).add(ee.Image(img))),
            # streakAccumulation --> yesterday's accum plus current 1/0, if in streak
            streakAccumulation.add(ee.Image(img).multiply(continueStreakImage)),
            # numRemaining --> minus 1
            numRemaining.add(-1)
          ]
        )

        return accumulator
  
    resultImageName = {
        'count': 'streakCountImage',
        'max': 'longestStreakImage',
        'accum': 'streakAccumulation'
    }
  
    streakData = imgs.iterate(
    # iterate over each image in the ImageCollection
    #   accumulate a stateful Dictionary of images
    
        doOne, ee.Dictionary.fromLists([
          'previousThresholdImage', 
          'currentStreakImage',
          'streakCountImage',
          'longestStreakImage',
          'streakAccumulation',
          'numRemaining'
        ], [
          ee.Image.constant(1),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          imgs.size().add(-1)
        ]
        )
    
    )
    return ee.Image(ee.Dictionary(streakData).get(resultImageName[resultType]))

In [55]:
NEX = ee.ImageCollection("NASA/NEX-GDDP")
PR_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/pr_99')
TASMAX_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmax_99')
TASMIN_01 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmin_01')
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
NOLEAP_MODELS = ['bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM-BGC', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC5', 'NorESM1-M']
START_YEAR = 2010
END_YEAR = 2099

def skipleap(d, model):
    # Checks whether d is Feb 29. If not, returns d. If it is and model in list of leap-intolerant models, returns March 1.
    if model in NOLEAP_MODELS and d.month == 2 and d.day == 29:
        return date(d.year, 3, 1)
    else:
        return d

In [60]:
for hazard in hazards:
    for year in range(START_YEAR + 1, END_YEAR + 1):
        for model in MODELS:
            data = NEX.filterMetadata('model', 'equals', model).filterDate(str(year) + '-01-01', str(year) + '-12-31').filterMetadata('scenario', 'equals', ['historical', 'rcp85'][(year > 2005) * 1]).select(hazard['variable'])
            img = hazard['definition'](data)
            task = ee.batch.Export.image.toAsset(**{
              'image': img.rename(hazard['definition'].__name__).set('model', model).set('year', year),
              'description': '{0}_{1}_{2}'.format(hazard['definition'].__name__, str(year), model),
              'assetId': 'users/tedwongwri/dataportal/magnitudes/{0}/{1}_{2}'.format(hazard['definition'].__name__, str(year), model),
              'region': WHOLE_GLOBE,
              'crs': 'EPSG:4326',
              'dimensions': '1440x720'
            })
            task.start()